In [16]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import time

import sys
sys.path.append('../')
from src.parameters import STR_CONN

In [13]:
def download_data_ticker(stock_symbol):
    ticker = yf.Ticker(stock_symbol)
    list_fields = ['symbol', 'country', 'industry', 'sector', 'fullTimeEmployees', 'ebitda', 'totalDebt', 'totalRevenue', 'debtToEquity', 'grossMargins', 'ebitdaMargins']
    ticker = pd.DataFrame.from_dict({k:v for k,v in ticker.info.items() if k in list_fields}, orient='index').T
    ticker.columns = ticker.columns.str.lower()
    return ticker

def download_data_stocks(stock_symbol, start_date, end_date):
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    stock_data['symbol'] = stock_symbol
    stock_data = stock_data.reset_index()
    stock_data.columns = stock_data.columns.str.lower()
    stock_data = stock_data.drop(['adj close'], axis=1)
    return stock_data

In [14]:
end_date = datetime.now()
start_date = end_date - timedelta(days=365*2)
# stock_symbol = 'GOOG'

In [15]:
stock_symbols = pd.read_excel('../data/tickers.xlsx')
stock_symbols = stock_symbols['Símbolo'].to_list()

In [21]:
stock_symbols = ['META']

In [22]:
bad_symbol = []
for stock_symbol in stock_symbols:
    try:
        ticker = download_data_ticker(stock_symbol)
        stock_data = download_data_stocks(stock_symbol, start_date, end_date)
        try:
            ticker.to_sql('stocks', con=STR_CONN, if_exists='append', index=False)
            stock_data.to_sql('stock_prices', con=STR_CONN, if_exists='append', index=False)
        except Exception as e:
            print('Not upload in database: ', stock_symbol)
            print(e)
    except Exception as e:
        print(stock_symbol, e)
        bad_symbol.append(stock_symbol)
    time.sleep(1)

[*********************100%***********************]  1 of 1 completed


In [23]:
bad_symbol

[]